<a href="https://colab.research.google.com/github/Jason-Durr/Projects-in-Machine-Learning/blob/main/HW/Homework2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Jason Durr

In [1]:
from google.colab import files
files.upload()


Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"jasondurr","key":"a288e7df4fa66f235cefea7adda21090"}'}

In [2]:
!mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json
!kaggle datasets download -d tobycrabtree/nfl-scores-and-betting-data
! unzip /content/nfl-scores-and-betting-data.zip



  0% 0.00/235k [00:00<?, ?B/s]
100% 235k/235k [00:00<00:00, 71.3MB/s]
Archive:  /content/nfl-scores-and-betting-data.zip
  inflating: nfl_stadiums.csv        
  inflating: nfl_teams.csv           
  inflating: spreadspoke.R           
  inflating: spreadspoke_scores.csv  


In [3]:
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score




In [4]:
data = pd.read_csv('spreadspoke_scores.csv')
teams = pd.read_csv('nfl_teams.csv')
df = data.dropna(subset=['team_favorite_id','weather_temperature'])
df = df[df['stadium_neutral'] == False]
df = df[df['schedule_week'] != 17]
df.weather_detail = df.weather_detail.fillna('Clear')
df.over_under_line = df.over_under_line.replace(' ',np.NaN)
df.weather_detail.unique()
X = pd.DataFrame()
X["Home_Favorite"] = np.where(df['team_home'].map(teams.set_index('team_name')['team_id']) == df['team_favorite_id'], 1, 0)
X["Temperature"] = (df.weather_temperature.to_numpy() - data.weather_temperature.median())/df.weather_temperature.apply(lambda x: float(x)).std()
X["Humidity"] = (df.weather_humidity.apply(lambda x: float(x)).to_numpy() - df.weather_humidity.apply(lambda x: float(x)).median())/df.weather_humidity.apply(lambda x: float(x)).std()
X["Wind"] = (df.weather_wind_mph.to_numpy() - data.weather_wind_mph.median())/df.weather_wind_mph.apply(lambda x: float(x)).std()
X["Over_Under"] = (df.over_under_line.apply(lambda x: float(x)).to_numpy() - df.over_under_line.apply(lambda x: float(x)).median())/df.over_under_line.apply(lambda x: float(x)).std()
X['Rain'] = df.weather_detail.apply(lambda x: int('Rain' in x)).to_numpy()
X['Snow'] = df.weather_detail.apply(lambda x: int('Snow' in x)).to_numpy()
X['Fog'] = df.weather_detail.apply(lambda x: int('Fog' in x)).to_numpy()
X['Playoff'] = df.schedule_playoff.apply(lambda x: int(x)).to_numpy()
X["Home_Win"] = np.where(df.score_home > df.score_away,1,0)
X = X.dropna()

In [5]:
Y = np.matrix(X['Home_Win']).T
x = X.drop('Home_Win',axis=1).to_numpy()

In [6]:
x

array([[ 1.        ,  1.1654317 ,  1.18492639, ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.84170067,  0.37418728, ...,  0.        ,
         0.        ,  0.        ],
       [ 1.        ,  1.10068549,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       ...,
       [ 1.        , -1.10068549,  1.55911367, ...,  0.        ,
         0.        ,  0.        ],
       [ 1.        , -0.51796964, -0.43655183, ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.12949241,  0.62364547, ...,  0.        ,
         0.        ,  1.        ]])

In [31]:

sample = np.random.choice(x.shape[0],size=x.shape[0]//5)
x_train = x[~sample,:]
x_test = x[sample,:]
y_train = Y[~sample]
y_test = Y[sample]

# Task 1 Implement Decision Tree Classifier


In [32]:
from sklearn import tree
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.model_selection import cross_val_score
from numpy import mean
from numpy import std
clf1 = tree.DecisionTreeClassifier(max_depth=3)
clf1 = clf1.fit(x_train, y_train)
accuracy_score(y_test,clf1.predict(x_test))
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
n_scores = cross_val_score(clf1, x_train, y_train, scoring='accuracy', cv=cv, n_jobs=-1, error_score='raise')
print('Depth 3 Accuracy: %.3f (%.3f)' % (mean(n_scores), std(n_scores)))

Depth 3 Accuracy: 0.671 (0.035)


In [36]:
clf2 = tree.DecisionTreeClassifier(max_depth=9)
clf2 = clf.fit(x_train, y_train)
accuracy_score(y_test,clf2.predict(x_test))
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
n_scores = cross_val_score(clf2, x_train, y_train, scoring='accuracy', cv=cv, n_jobs=-1, error_score='raise')
print('Depth 9 Accuracy: %.3f (%.3f)' % (mean(n_scores), std(n_scores)))

Depth 9 Accuracy: 0.679 (0.036)


In [35]:
clf3 = tree.DecisionTreeClassifier(max_depth=100)
clf3 = clf.fit(x_train, y_train)
accuracy_score(y_test,clf3.predict(x_test))
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
n_scores = cross_val_score(clf3, x_train, y_train, scoring='accuracy', cv=cv, n_jobs=-1, error_score='raise')
print('Depth 100 Accuracy: %.3f (%.3f)' % (mean(n_scores), std(n_scores)))

Depth 100 Accuracy: 0.679 (0.036)


In [34]:
clf4 = tree.DecisionTreeClassifier(max_depth=1)
clf4 = clf.fit(x_train, y_train)
accuracy_score(y_test,clf4.predict(x_test))
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
n_scores = cross_val_score(clf4, x_train, y_train, scoring='accuracy', cv=cv, n_jobs=-1, error_score='raise')
print('Depth 1 Accuracy: %.3f (%.3f)' % (mean(n_scores), std(n_scores)))

Depth 1 Accuracy: 0.679 (0.036)


### For Comparison The Accuracy of just picking the favorite


In [33]:
print("Always Pick the Favorite Accuracy:",accuracy_score(y_train,x_train[:,0]))

Always Pick the Favorite Accuracy: 0.6786283891547049


From modifying the max_depth parameter it forces the tree to prune itself which increases the bias but reduces the overfitting.  This seen when comparing a max depth of 9 and 100 with that of 3 and we see that they preform worse out of sample.  We also see that by going too far the other way it increases the bias to a point where it is the same as our nieve attempt of just picking the favorite everytime and ignoring the rest of the data.  It also shows that the standard deviation of accuracy increases when we increase the allowed size of the tree because there is mor variance

# Task 2 Implement Bagging and Boosting and Use k-fold Cross Validation

## Bagging

In [37]:
from sklearn.ensemble import BaggingClassifier
model = BaggingClassifier(n_estimators=500)
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
n_scores = cross_val_score(model, x_train, y_train, scoring='accuracy', cv=cv, n_jobs=-1, error_score='raise')
# report performance
print('Bagging Accuracy: %.3f (%.3f)' % (mean(n_scores), std(n_scores)))

Bagging Accuracy: 0.657 (0.031)


## Boosting

In [38]:
from sklearn.ensemble import AdaBoostClassifier
model = AdaBoostClassifier(n_estimators=4, random_state=0, algorithm='SAMME')
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
n_scores = cross_val_score(model, x_train, y_train, scoring='accuracy', cv=cv, n_jobs=-1, error_score='raise')
# report performance
print('Boosting Accuracy: %.3f (%.3f)' % (mean(n_scores), std(n_scores)))

Boosting Accuracy: 0.679 (0.036)


This expirament shows that bagging and boosting both helped to achieve better accuracies the almost all of the regular decision trees and this is because it has reduced the variance of the model.  Because boosting focuses on the points that are misclassified it is able to achieve a model with a higher accuracy than bagging but ultimately what this shows is that this data does not contain enough information to beat the naive model of just choosing the favorite to win.

# Task 3 Compare the effectiveness of the three models implemented above


The main metric I have been using to compare the models is the accuracy of prediction compared to that of just selecting the favorite.  I chose this metric because when predicting the outcome of games the goal is do better than the odds setters as predicting the game so just matching their performance means there is no new knowledge being learned.  The reason I did not use a metric like false positives/ false negative or precision/sensitivity is because in this scenario all cases are weighted equally. This is because predicting the home team loses when they win or predicting them to win when they loose has the same consiquences and so there is no need to weight these scenarios differently.  If instead we tried to minimize false negaitves for example and lean toward only predicting they will loose if we are more sure that they will, this might lower the overall accuracy of the model because we are increasing the bias in order to reduce only a certain number of mistakes.  We see this by comparing the confusion matrices of the 4 different trees with different stopping parameters and note that model 1 has a worse accuracy but a better false negative score than the rest of the models which all just pick the team that is favorited.

In [39]:
from sklearn.metrics import confusion_matrix

print(confusion_matrix(y_test,clf1.predict(x_train)))
print(confusion_matrix(y_test,clf2.predict(x_train)))
print(confusion_matrix(y_test,clf3.predict(x_train)))
print(confusion_matrix(y_test,clf4.predict(x_train)))

[[176 336]
 [218 524]]
[[178 334]
 [219 523]]
[[178 334]
 [219 523]]
[[178 334]
 [219 523]]
